
The general recipe is a short list of four main steps:

1.   Compose a function to **read** input data and prepare a Tensorflow Dataset;
2.   Define a **scoring** function that, given a (set of) query-document feature vector(s), produces a score indicating the query's level of relevance to the document;
3.   Create a **loss** function that measures how far off the produced scores from step (2) are from the ground truth; and,
4.   Define evaluation **metrics**.

## Imports

In [20]:
import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd

tf.enable_eager_execution()
tf.executing_eagerly()

True

## Constants

In [21]:
_TRAIN_DATA_PATH="../data/interim/train_sample_100.libsvm"
_TEST_DATA_PATH="../data/interim/val_sample_100.libsvm"

_LOSS="pairwise_logistic_loss"
# _LOSS="sigmoid_cross_entropy_loss"

_LIST_SIZE=10

_NUM_FEATURES=4

_BATCH_SIZE=1000
_HIDDEN_LAYER_DIMS=["20", "10"]

#_OUT_DIR = "../models/tfranking/"

## Function to read in data and form tensorflow dataset

Out train and test dataset is in the lib svm format which is normally used for Support Vector Machines. 

In [22]:
fo = open(_TRAIN_DATA_PATH)
i=0
for f in fo:
    if i != 10:
        print(f)
    else:
        break
    i+=1

100 qid:10 1:4 2:47429 3:4604 4:14100

100 qid:10 1:11 2:47796 3:7234 4:3000

100 qid:10 1:8 2:49758 3:6878 4:5500

100 qid:10 1:3 2:47429 3:4604

100 qid:10 1:2 2:49067 3:6345 4:3100

0 qid:10 1:1 2:48995 3:7396 4:3200

100 qid:21 1:4 2:7667 3:1229 4:2100

100 qid:21 1:3 2:7667 3:929

0 qid:21 1:2 2:6157 3:1289 4:300

100 qid:25 1:4 2:3898 3:669 4:1300



In this example they first number shows the target

1 = Important 

0 = Not important

qid: Describes which lines belong together

E.g. query 10 had 6 suggestions for plans and just one is important. Here we would take the suggested transport mode from this plan. 

### Input Pipeline

The first step to construct an input pipeline that reads your dataset and produces a `tensorflow.data.Dataset` object. In this example, we will invoke a LibSVM parser that is included in the `tensorflow_ranking.data` module to generate a `Dataset` from a given file.

We parameterize this function by a `path` argument so that the function can be used to read both training and test data files.

 Read tf DataSet
 
 Dic for feature

In [23]:
def input_fn(path):
    train_dataset = tf.data.Dataset.from_generator(
      tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
      output_types=(
          {str(k): tf.float32 for k in range(1,_NUM_FEATURES+1)},
          tf.float32
      ),
      output_shapes=(
          {str(k): tf.TensorShape([_LIST_SIZE, 1])
            for k in range(1,_NUM_FEATURES+1)},
          tf.TensorShape([_LIST_SIZE])
      )
    )

    train_dataset = train_dataset.shuffle(1000).repeat().batch(_BATCH_SIZE)
    return train_dataset.make_one_shot_iterator().get_next()

In [24]:
train_dataset = tf.data.Dataset.from_generator(
      tfr.data.libsvm_generator(_TRAIN_DATA_PATH, _NUM_FEATURES, _LIST_SIZE),
      output_types=(
          {str(k): tf.float32 for k in range(1,_NUM_FEATURES+1)},
          tf.float32
      ),
      output_shapes=(
          {str(k): tf.TensorShape([_LIST_SIZE, 1])
            for k in range(1,_NUM_FEATURES+1)},
          tf.TensorShape([_LIST_SIZE])
      )
    )

train_dataset

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


<DatasetV1Adapter shapes: ({1: (10, 1), 2: (10, 1), 3: (10, 1), 4: (10, 1)}, (10,)), types: ({1: tf.float32, 2: tf.float32, 3: tf.float32, 4: tf.float32}, tf.float32)>

In [25]:
a=train_dataset.make_one_shot_iterator().get_next()

In [26]:
a[0]

{'1': <tf.Tensor: id=83, shape=(10, 1), dtype=float32, numpy=
 array([[ 4.],
        [ 8.],
        [ 3.],
        [11.],
        [ 1.],
        [ 2.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.]], dtype=float32)>,
 '2': <tf.Tensor: id=84, shape=(10, 1), dtype=float32, numpy=
 array([[47429.],
        [49758.],
        [47429.],
        [47796.],
        [48995.],
        [49067.],
        [    0.],
        [    0.],
        [    0.],
        [    0.]], dtype=float32)>,
 '3': <tf.Tensor: id=85, shape=(10, 1), dtype=float32, numpy=
 array([[4604.],
        [6878.],
        [4604.],
        [7234.],
        [7396.],
        [6345.],
        [   0.],
        [   0.],
        [   0.],
        [   0.]], dtype=float32)>,
 '4': <tf.Tensor: id=86, shape=(10, 1), dtype=float32, numpy=
 array([[14100.],
        [ 5500.],
        [    0.],
        [ 3000.],
        [ 3200.],
        [ 3100.],
        [    0.],
        [    0.],
        [    0.],
        [    0.]], dtype=float32)>}

In [27]:
a[1]

<tf.Tensor: id=87, shape=(10,), dtype=float32, numpy=
array([100., 100., 100., 100.,   0., 100.,  -1.,  -1.,  -1.,  -1.],
      dtype=float32)>

## Estimator Creation


Next, we turn to the scoring function which is arguably at the heart of a TF Ranking model. The idea is to compute a relevance score for a (set of) query-document pair(s). The TF-Ranking model will use training data to learn this function.

Here we formulate a scoring function using a feed forward network. The function takes the features of a single example (i.e., query-document pair) and produces a relevance score

In [28]:
def example_feature_columns():
    """Returns the example feature columns."""
    feature_names = [
      "%d" % (i + 1) for i in range(0, _NUM_FEATURES)
    ]
    return {
      name: tf.feature_column.numeric_column(
          name, shape=(1,), default_value=0.0) for name in feature_names
    }

def make_score_fn():
        """Returns a scoring function to build `EstimatorSpec`."""
        
        def _score_fn(context_features, group_features, mode, params, config):
            """Defines the network to score a documents."""
            del params
            del config
            # Define input layer.
            example_input = [
                tf.layers.flatten(group_features[name])
                for name in sorted(example_feature_columns())
            ]
            input_layer = tf.concat(example_input, 1)

            cur_layer = input_layer
            for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
                cur_layer = tf.layers.dense(
                  cur_layer,
                  units=layer_width,
                  activation="tanh")

            logits = tf.layers.dense(cur_layer, units=1)
            return logits
        return _score_fn

In [29]:
def eval_metric_fns():
    """Returns a dict from name to metric functions.

    This can be customized as follows. Care must be taken when handling padded
    lists.

    def _auc(labels, predictions, features):
    is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
    clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
    clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
    return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
    metric_fns["auc"] = _auc

    Returns:
    A dict mapping from metric name to a metric function with above signature.
    """
    metric_fns = {}
    metric_fns.update({
      "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
          tfr.metrics.RankingMetricKey.NDCG, topn=topn)
      for topn in [1, 3, 5, 10]
    })

    return metric_fns

In [30]:
def get_estimator(hparams):
    """Create a ranking estimator.

    Args:
    hparams: (tf.contrib.training.HParams) a hyperparameters object.

    Returns:
    tf.learn `Estimator`.
    """
    def _train_op_fn(loss):
        """Defines train op used in ranking head."""
        return tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.train.get_global_step(),
            learning_rate=hparams.learning_rate,
            optimizer="Adagrad")

    ranking_head = tfr.head.create_ranking_head(
      loss_fn=tfr.losses.make_loss_fn(_LOSS),
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)

    return tf.estimator.Estimator(
      model_fn=tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(),
          group_size=1,
          transform_fn=None,
          ranking_head=ranking_head),
        params=hparams)

In [31]:
hparams = tf.contrib.training.HParams(learning_rate=0.05)
ranker = get_estimator(hparams)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp8it4iksd', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe19c2de390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [32]:
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


/home/sandro/anaconda3/envs/tf_ranking/lib/python3.7/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp8it4iksd/model.ckpt.
INFO:tensorflow:loss = 74.1089, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmp8it4iksd/model.ckpt.
INFO:tensorflow:Loss for final step: 61.284252.


In [33]:
train

array([[1.000e+00, 1.285e+03, 1.088e+03, 2.000e+02],
       [3.000e+00, 9.160e+02, 1.880e+02, 0.000e+00],
       [5.000e+00, 9.230e+02, 8.590e+02, 0.000e+00],
       ...,
       [1.000e+00, 6.102e+03, 2.547e+03, 2.000e+02],
       [1.000e+00, 4.217e+03, 1.926e+03, 2.000e+02],
       [3.000e+00, 7.594e+03, 1.181e+03, 0.000e+00]])

In [36]:
predictions = ranker.predict(input_fn=lambda: ("../data/interim/val_sample_100_wo_target.libsvm.txt", None))

In [37]:
for i in predictions:
    print(i)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.


AttributeError: 'str' object has no attribute 'items'

In [25]:
def ltr_to_submission(df, features, ranker, path):
    features = features + ['sid']
    
    preds = ranker.predict(input_fn=lambda: input_fn(path))
    import itertools
    import numpy as np
    # Not sure how to get all preds because it runs infinit
    # So I take all till list size
    preds_slice = itertools.islice(preds, len(df)) 
    count=0
    a = np.zeros((len(df), _LIST_SIZE))

    for i in preds_slice:
        a[count]=i
        count+=1
        
        
    test_X = df[features]
    
    # Assign prediction vals to df
    # Tried with a or a sum for all features
    
    # --- Sum of the first n feature values --- 
    # a = a[:,0:_NUM_FEATURES]
    # a = a.sum(axis=1)
    
    # test_X = test_X.assign(yhat = a)
    
     # --- Just the feature prio where the transport mode is in --- 
    
    test_X = test_X.assign(yhat = a[:,0])
    
    df_end = pd.DataFrame(columns=['yhat'], index=df.sid.unique())

    df_end = test_X.sort_values(['sid', 'yhat'], ascending=False).groupby('sid').first()[[
        'yhat', 'transport_mode'
    ]]
    # return df_end
    
    from sklearn.metrics import f1_score
    score = f1_score(df.groupby("sid").first()['click_mode'], df_end.transport_mode, average='weighted')
    print('F1 Score is: {}'.format(score))
    
    return df_end

In [26]:
df_train_train = pd.read_pickle("../data/interim/train_row.pickle")

In [27]:
df_train_test = pd.read_pickle("../data/interim/val_row.pickle")

In [28]:
df_train_test.head()

,sid,click_mode,distance_plan,eta,price,transport_mode,plan_time,pid,req_time,o_long,...,is_holiday,max_temp,min_temp,wind,dy,dyq,q,qdy,xq,xydy
1258547,697184.0,1.0,7594.0,1361.0,2200,4.0,2018-11-15 05:16:06,132790.0,2018-11-15 05:16:06,116.42,...,0,8,1,34,0,0,0,0,0,1
1258549,697184.0,1.0,3932.0,3605.0,0,5.0,2018-11-15 05:16:06,132790.0,2018-11-15 05:16:06,116.42,...,0,8,1,34,0,0,0,0,0,1
272907,673286.0,2.0,9558.0,2875.0,400,2.0,2018-11-15 05:16:52,0.0,2018-11-15 05:16:52,116.37,...,0,8,1,34,0,0,0,0,0,1
272911,673286.0,2.0,8900.0,3373.0,400,1.0,2018-11-15 05:16:52,0.0,2018-11-15 05:16:52,116.37,...,0,8,1,34,0,0,0,0,0,1
272909,673286.0,2.0,7916.0,2390.0,0,6.0,2018-11-15 05:16:52,0.0,2018-11-15 05:16:52,116.37,...,0,8,1,34,0,0,0,0,0,1


In [29]:
features = pd.read_pickle("../data/interim/features_row_sample.pickle")

In [30]:
df_preds = ltr_to_submission(df_train_test, features, ranker, _TEST_DATA_PATH)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpw4umrvzi/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
F1 Score is: 0.317192514928933


In [241]:
df_end = test_X[msk][['sid', 'transport_mode']]

In [265]:
test_X.groupby('sid', sort=False)['yhat'].transform(max)

143113     0.509255
143110     0.509255
143112     0.509255
143109     0.509255
459059     0.509255
459057     0.509255
459061     0.509255
459062     0.509255
459058     0.509255
459060     0.509255
909545     0.509255
909546     0.509255
909548     0.509255
909547     0.509255
241776    -1.318469
241774    -1.318469
241775    -1.318469
241773    -1.318469
241777    -1.318469
1270163    0.509255
1270164    0.509255
1270165    0.509255
1270168    0.509255
1270167    0.509255
1270166    0.509255
316955     0.509255
316959     0.509255
316957     0.509255
316958     0.509255
316956     0.509255
             ...   
1525598    0.000000
1525603    0.000000
1525602    0.000000
1525599    0.000000
1525600    0.000000
45680      0.000000
45682      0.000000
45681      0.000000
45678      0.000000
45679      0.000000
45683      0.000000
1449538    0.000000
1449537    0.000000
1449539    0.000000
1449535    0.000000
1449536    0.000000
1326911    0.000000
1326910    0.000000
1326912    0.000000


In [277]:
test_X['new']=test_X.groupby('sid')['yhat'].transform(max)

In [289]:
test_X.sort_values(['sid', 'yhat'], ascending=False).groupby('sid').first()

(129542, 6)

In [242]:
df_end.shape

(600983, 2)

In [75]:
preds = ranker.predict(input_fn=lambda: input_fn(_TRAIN_DATA_PATH))
import itertools
import numpy as np
# Not sure how to get all preds because it runs infinit
# So I take all till list size
preds_slice = itertools.islice(preds, len(df_train_test)) 
count=0
a = np.zeros((len(df_train_test), _LIST_SIZE))

for i in preds_slice:
    a[count]=i
    count+=1

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Use groupwise dnn v2.
MAKE SCORE FUNCTION:
[<tf.Tensor 'groupwise_dnn_v2/group_score/flatten/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_1/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_2/Reshape:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'groupwise_dnn_v2/group_score/flatten_3/Reshape:0' shape=(?, 1) dtype=float32>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp_09c8s1t/model.ckpt-1100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [93]:
a=a[:,0:3]
a = a.sum(axis=1)

In [101]:
test_X = df_train_test[[
       'sid',
        'transport_mode',
        'distance_plan',
        'eta', 
        'price'
    ]]
    
# Assign prediction vals to df
test_X = test_X.assign(yhat = a)

df_end = pd.DataFrame(columns=['yhat'], index=df_train_test.sid.unique())

df_end = test_X.sort_values(['sid', 'yhat'], ascending=False).groupby('sid').first()[[
    'yhat', 'transport_mode'
]]

In [102]:
from sklearn.metrics import f1_score

f1_score(df_train_train.groupby("sid").first()['click_mode'], df_end.transport_mode, average='weighted')

0.3075881119766427

# Try datasets from numpy

In [2]:
df_train_train = pd.read_pickle("../data/interim/train_row.pickle")
features = pd.read_pickle("../data/interim/features_row_sample.pickle")
train = df_train_train[features].values

In [3]:
df_train_train[features].head()

,transport_mode,distance_plan,eta,price
1027273,1.0,1285.0,1088.0,200
1027271,3.0,916.0,188.0,0
1027270,5.0,923.0,859.0,0
1027272,6.0,923.0,279.0,0
1814953,2.0,35361.0,5259.0,700


In [6]:
labels = df_train_train.click_mode.values

In [9]:
tf_data = tf.data.Dataset.from_tensor_slices((train, labels))

In [12]:
def input_fn(path):
    train_dataset = tf.data.Dataset.from_tensor_slices((train, labels))

    train_dataset = train_dataset.shuffle(1000).repeat().batch(_BATCH_SIZE)
    return train_dataset.make_one_shot_iterator().get_next()